Annotating

In [1]:
# import cv2
# import os
# import glob
# import pandas as pd
# import numpy as np

# annotations = pd.read_csv('/home/akhan/annotations.txt',
#                           delim_whitespace=True,
#                           names=['start', 'end', 'duration', 'class', 'filename'])

# annotations['duration'] = 0

# renamer = lambda x: x.split('.')[0] + '_crop.mp4'

# annotations.filename = annotations.filename.apply(renamer)

# for path, subdirs, files in os.walk('/home/akhan/Videos/data/videos'):
#     for name in files:
#         if name.endswith('crop.mp4'):
#             cap = cv2.VideoCapture(os.path.join(path, name))
#             fps = cap.get(cv2.CAP_PROP_FPS)  # OpenCV2 version 2 used "CV_CAP_PROP_FPS"
#             frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
#             duration = np.round(frame_count / fps, 3)
#             annotations.loc[annotations['filename'] == name, 'duration'] = np.round(duration, 3)

# annotations.loc[annotations['class'] == 'both_hand', 'class'] = 'BothHand'
# annotations.loc[annotations['class'] == 'single_hand', 'class'] = 'SingleHand'

# annotations.to_csv('/home/akhan/ActionDetection-AFSD/thumos_annotations/custom_annotations.csv',
#                   header=False, index=False)

Splitting videos

In [2]:
# video_names = annotations.filename.unique()

# np.random.shuffle(video_names)

# test = video_names[57::]
# train = video_names[:57]

# from shutil import copyfile

# %cd /home/akhan/ActionDetection-AFSD/datasets/custom_dataset/

# for path, subdirs, files in os.walk('/home/akhan/ActionDetection-AFSD/datasets/custom_dataset/videos/'):
#     rename = os.path.join(path) + '_crop.mp4'
#     if os.path.basename(rename) in test:
#         cropvideo = [x for x in files if x.endswith('_crop.mp4')][0]
#         copyfile(os.path.join(path, cropvideo),
#                   os.path.join(os.getcwd(), 'test', cropvideo))
#     elif os.path.basename(rename) in train:
#         cropvideo = [x for x in files if x.endswith('_crop.mp4')][0]
#         copyfile(os.path.join(path, cropvideo),
#                   os.path.join(os.getcwd(), 'validation', cropvideo))

Generating `{Val, Test}_annotation_ours.csv`

In [1]:
import pandas as pd
import os
import numpy as np
import glob 
import cv2

In [2]:
custom_annotations = pd.read_csv('/home/akhan/ActionDetection-AFSD/thumos_annotations/custom_annotations.csv',
                                 names=['start', 'end', 'total_duration', 'type', 'video'])

In [3]:
custom_annotations.iloc[:,[0,1]] = custom_annotations.iloc[:, [0,1]].round(2)

Need to generate `startFrame` and `endFrame`

In [4]:
videodir = '/home/akhan/ActionDetection-AFSD/datasets/custom_dataset/'
testdir = os.path.join(videodir, 'test')
valdir = os.path.join(videodir, 'validation')

In [5]:
valvids = glob.glob(os.path.join(valdir, "*.mp4"))
testvids = glob.glob(os.path.join(testdir, "*.mp4"))

In [6]:
def time2frame(x, fps=30.0):
    return np.round(x * fps, decimals=0).astype(int)


def type2idx(x):
    if x == 'BothHand':
        return 23
    elif x == 'SingleHand':
        return 24

In [7]:
start_end_frames = custom_annotations.iloc[:,[0,1]].apply(time2frame)
start_end_frames.columns = ['startFrame', 'endFrame']

In [8]:
idx = custom_annotations['type'].apply(type2idx)
idx.name = 'type_idx'

In [9]:
custom_annotations = custom_annotations.assign(**pd.DataFrame(idx))

In [10]:
custom_annotations = custom_annotations.assign(**start_end_frames)

In [11]:
columns = ['video', 'type', 'type_idx', 'start', 'end', 'startFrame', 'endFrame']

In [33]:
new_custom_annotations = custom_annotations[columns]

In [34]:
new_custom_annotations.head()

,video,type,type_idx,start,end,startFrame,endFrame
0,servings00_crop.mp4,BothHand,23,0.00,1.65,0,50
1,servings00_crop.mp4,BothHand,23,2.31,4.52,69,136
2,servings00_crop.mp4,BothHand,23,5.53,7.93,166,238
3,servings00_crop.mp4,BothHand,23,8.48,10.23,254,307
4,servings00_crop.mp4,SingleHand,24,11.23,13.98,337,419


In [24]:
def extensionless(x: str):
    assert '.' in x
    return os.path.basename(x.split('.')[0])

In [38]:
basename_val = list(map(os.path.basename, valvids))
basename_test = list(map(os.path.basename, testvids))

In [47]:
test_Annotation = new_custom_annotations.loc[new_custom_annotations['video'].isin(basename_test), :]

In [48]:
val_Annotation = new_custom_annotations.loc[new_custom_annotations['video'].isin(basename_val), :]

In [51]:
test_Annotation['video'] = test_Annotation['video'].apply(extensionless)
val_Annotation['video'] = val_Annotation['video'].apply(extensionless)

<ipython-input-51-44ceef17ba08>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_Annotation['video'] = test_Annotation['video'].apply(extensionless)
<ipython-input-51-44ceef17ba08>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_Annotation['video'] = val_Annotation['video'].apply(extensionless)


In [53]:
val_Annotation.to_csv('/home/akhan/ActionDetection-AFSD/thumos_annotations/servings/val_Annotation.csv',
                     index=False)

In [54]:
test_Annotation.to_csv('/home/akhan/ActionDetection-AFSD/thumos_annotations/servings/test_Annotation.csv',
                     index=False)